<a href="https://colab.research.google.com/github/Eddycrack864/Demucs-for-Colab/blob/main/Demucs_for_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demucs (All Models) for Google Colab
This is a Demucs colab that has all the models from Demucs V1 to Demucs V4 with customizable parameters and batch conversion.
### Colab created by [Not Eddy (Spanish Mod)](http://discord.com/users/274566299349155851) in [AI HUB](https://discord.gg/aihub) server.
### Thanks to [Alexandre Défossez](https://ai.honu.io/) for the repo.

In [ ]:
#@markdown #Installation
from IPython.display import clear_output
#@markdown Select the version of Demucs you are going to use. (To change to the other version you must restart the environment)
demucs_version = "V3 - V4" #@param ["V3 - V4", "V1 - V2"]
if demucs_version == 'V3 - V4':
  !pip install demucs==4.0.1
else:
  !pip install demucs==2.0.3
#Drive
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print("Installation Completed!!!")

In [ ]:
#@markdown #Separate! Only for Demucs V3 - V4
#@markdown Input path for audio files folder:
in_path = '/content/drive/MyDrive/Separar' #@param {type:"string"}
#@markdown Output path for audio files folder:
out_path = '/content/drive/MyDrive/Vocales' #@param {type:"string"}
#@markdown ---
#@markdown Select the Demucs model:
model = "htdemucs_ft" #@param ["htdemucs_ft", "htdemucs", "hdemucs_mmi", "htdemucs_6s", "mdx", "mdx_q", "mdx_extra", "mdx_extra_q", "UVR Model", "repro_mdx_a", "repro_mdx_a_time_only", "repro_mdx_a_hybrid_only"]
#@markdown ---
#@markdown Select the output format for the audio file:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Select the bitrate (It only applies if the audio output format is mp3)
mp3_bitrate = "320" #@param ["320", "256", "192", "128", "96", "64", "32", "16", "8"]
#@markdown Select the bit depth (Only applicable if the audio output format is wav)
wav_bit_depth = "float32" #@param ["int24", "float32"]
#@markdown ---
#@markdown Controls the amount of overlap between prediction windows. Default is 0.25 (i.e. 25%) which is probably fine.
overlap = True #@param {type:"boolean"}
overlap_value = 0.25 #@param {type:"slider", min:0.1, max:0.99, step:0.01}
#@markdown Performs multiple predictions with random shifts (a.k.a the shift trick) of the input and average them. **Increase separation time** but improves quality for Demucs.
shifts = True #@param {type:"boolean"}
shifts_value = 10 #@param {type:"slider", min:0, max:10, step:1}
#@markdown Strategy for avoiding clipping: rescaling entire signal if necessary (rescale) or hard clipping (clamp).
clipmode = True #@param {type:"boolean"}
clipmode_value = "rescale" #@param ["rescale", "clamp"]
#@markdown ---
#@markdown Write `None` for 4 stems or write: `drums` or `vocals` or `other` or `bass` for 2 stems.
two_stems = 'None' #@param {type:"string"}

extensions = ["mp3", "wav", "ogg", "flac"] #Allowed extensions

import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

#Separation
def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if output_format == 'wav':
        cmd += [f"--{wav_bit_depth}"]
    if output_format == 'flac':
        cmd += ["--flac"]
    if output_format == 'mp3':
        cmd += ["--mp3", f"--mp3-bitrate={mp3_bitrate}"]
    if overlap:
        cmd += ["--overlap", str(overlap_value)]
    if shifts:
        cmd += [f"--shifts={shifts_value}"]
    if clipmode:
        cmd += ["--clip-mode", f"{clipmode_value}"]
    if two_stems != 'None':
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")

#separate trigger
separate()

In [ ]:
#@markdown #Separate! Only for Demucs V1 -V2
#@markdown Input path for audio files folder:
in_path = '/content/drive/MyDrive/Separar' #@param {type:"string"}
#@markdown Output path for audio files folder:
out_path = '/content/drive/MyDrive/Vocales' #@param {type:"string"}
#@markdown ---
#@markdown Select the Demucs model:
model = "demucs" #@param ["demucs", "demucs_extra", "demucs48_hq", "tasnet", "tasnet_extra", "demucs_unittest"]
#@markdown ---
#@markdown Select the output format for the audio file:
output_format = "wav" #@param ["wav", "mp3"]
#@markdown Select the bitrate (It only applies if the audio output format is mp3)
mp3_bitrate = "320" #@param ["320", "256", "192", "128", "96", "64", "32", "16", "8"]
#@markdown Select the bit depth (Only applicable if the audio output format is wav)
wav_bit_depth = "float32" #@param ["int16", "float32"]
#@markdown ---
#@markdown Controls the amount of overlap between prediction windows. Default is 0.25 (i.e. 25%) which is probably fine.
overlap = True #@param {type:"boolean"}
overlap_value = 0.25 #@param {type:"slider", min:0.1, max:0.99, step:0.01}
#@markdown Performs multiple predictions with random shifts (a.k.a the shift trick) of the input and average them. **Increase separation time** but improves quality for Demucs.
shifts = True #@param {type:"boolean"}
shifts_value = 10 #@param {type:"slider", min:0, max:10, step:1}

extensions = ["mp3", "wav", "ogg", "flac"] #Allowed extensions

import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

#Separation
def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if output_format == 'wav':
        cmd += [f"--{wav_bit_depth}"]
    if output_format == 'mp3':
        cmd += ["--mp3", f"--mp3-bitrate={mp3_bitrate}"]
    if overlap:
        cmd += ["--overlap", str(overlap_value)]
    if shifts:
        cmd += [f"--shifts={shifts_value}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")

#Separate trigger
separate()